In [1]:
import pandas as pd

coord_file_path = 'adk-lakes-centroids-centerextracted.csv'
df_coord = pd.read_csv(coord_file_path)
df_coord.dropna(inplace=True)
df_coord

,PERMANENT_ID,GNIS_Name,AreaSqKm,Elevation,ReachCode,FType,NHDPlusID,area_ha,Lon,Lat
2,89362545,Challis Pond,0.065000,350.5,2.020000e+12,390,1.000000e+13,6.500000,-73.670123,43.984701
3,89361919,Bradley Pond,0.039364,0.0,2.020000e+12,390,1.000000e+13,3.936431,-74.110881,44.096109
27,47722313,Warner Lake,0.026000,0.0,2.020000e+12,390,1.000000e+13,2.600000,-74.563007,43.429974
29,47722092,Charley Lake,0.154000,0.0,2.020000e+12,390,1.000000e+13,15.400000,-74.318671,43.441170
45,89362163,Triangle Pond,0.021000,375.0,2.020000e+12,390,1.000000e+13,2.100000,-73.650409,44.039809
...,...,...,...,...,...,...,...,...,...,...
11842,132876274,Twin Ponds,0.091953,533.4,4.300000e+12,390,6.000000e+13,9.195258,-73.965560,44.675018
11851,132876003,Lower Chateaugay Lake,2.218336,399.0,4.300000e+12,390,6.000000e+13,221.800000,-74.018502,44.819012
11877,132876199,Mountain Pond,0.034300,0.0,4.300000e+12,390,6.000000e+13,3.430035,-74.042438,44.721654
11885,132876070,Bradley Pond,0.464830,0.0,4.300000e+12,390,6.000000e+13,46.480000,-73.887708,44.753069


In [2]:
import ee
import pandas as pd
import math  # Import the math module

# ee.Authenticate()

# Initialize Earth Engine
ee.Initialize()

# Define Landsat paths and rows of interest
paths = [14]
rows = [29, 30]

# Merge paths and rows into a single list of tuples
path_row_combinations = [(path, row) for path in paths for row in rows]

def get_s2_sr_cld_col(img,buffered_point):
    # Import and filter Landsat cloudless image collection
    # print("Type of 'img' inside get_s2_sr_cld_col function:", type(img))

    # Get the first image from the Landsat cloudless image collection
    s2_cloudless_img = ee.ImageCollection('LANDSAT/LC08/C02/T1').select('B1','B2','B3','B4','B5','B6','B7','SZA', 'VZA', 'VAA','SAA').first()

    return s2_cloudless_img


# MAIN Atmospheric Correction

def atm_corr(img):
    # print("Type of 'img' inside atm_corr function:", type(img))


    footprint = img.geometry()

    # DEM
    dem = ee.Image('USGS/SRTMGL1_003').clip(footprint)
    DEM_OLI = ee.Image(1);


    # ozone
    # DU_OLI = ee.Image(ozone.filterBounds(footprint).filter(ee.Filter.calendarRange(startMonth, endMonth, 'month')).filter(ee.Filter.calendarRange(startYear, endYear, 'year')).mean())

    DU_OLI = ee.Image(300); # ozone @ sea level

    # Julian Day
    imgDate_OLI = ee.Date(img.get('system:time_start'))
    FOY_OLI = ee.Date.fromYMD(imgDate_OLI.get('year'),1,1)
    JD_OLI = imgDate_OLI.difference(FOY_OLI,'day').int().add(1)

    # Earth-Sun distance
    d_OLI = ee.Image.constant(img.get('EARTH_SUN_DISTANCE'))

    # Sun elevation
    SunEl_OLI = ee.Image.constant(img.get('SUN_ELEVATION'))

    # Sun azimuth
    SunAz_OLI = img.select('SAA').multiply(ee.Image(0.01))

    # Satellite zenith
    SatZe_OLI = img.select('VZA').multiply(ee.Image(0.01))
    cosdSatZe_OLI = SatZe_OLI.multiply(ee.Image(math.pi).divide(180)).cos()
    sindSatZe_OLI = SatZe_OLI.multiply(ee.Image(math.pi).divide(180)).sin()

    

    # Satellite azimuth
    SatAz_OLI = img.select('VAA').multiply(ee.Image(0.01))

    # Sun zenith
    SunZe_OLI = img.select('SZA').multiply(ee.Image(0.01))
    cosdSunZe_OLI = SunZe_OLI.multiply(ee.Image.constant(math.pi).divide(ee.Image.constant(180))).cos() # in degrees
    sindSunZe_OLI = SunZe_OLI.multiply(ee.Image.constant(math.pi).divide(ee.Image.constant(180))).sin() # in degrees

 # Relative azimuth
    RelAz_OLI = ee.Image(SunAz_OLI)
    cosdRelAz_OLI = RelAz_OLI.multiply(ee.Image.constant(math.pi).divide(ee.Image(180))).cos()
    

    # Pressure calculation
    P_OLI = ee.Image(101325).multiply(ee.Image(1).subtract(ee.Image(0.0000225577).multiply(DEM_OLI)).pow(5.25588)).multiply(0.01)
    Po_OLI = ee.Image(1013.25)

    # Radiometric Calibration
    # define bands to be converted to radiance
    bands_OLI = ['B1','B2','B3','B4','B5','B6','B7']

    # radiance_mult_bands
    rad_mult_OLI = ee.Image(ee.Array([ee.Image(img.get('RADIANCE_MULT_BAND_1')),
                        ee.Image(img.get('RADIANCE_MULT_BAND_2')),
                        ee.Image(img.get('RADIANCE_MULT_BAND_3')),
                        ee.Image(img.get('RADIANCE_MULT_BAND_4')),
                        ee.Image(img.get('RADIANCE_MULT_BAND_5')),
                        ee.Image(img.get('RADIANCE_MULT_BAND_6')),
                        ee.Image(img.get('RADIANCE_MULT_BAND_7'))])).toArray(1)

    # radiance add band
    rad_add_OLI = ee.Image(ee.Array([ee.Image(img.get('RADIANCE_ADD_BAND_1')),
                        ee.Image(img.get('RADIANCE_ADD_BAND_2')),
                        ee.Image(img.get('RADIANCE_ADD_BAND_3')),
                        ee.Image(img.get('RADIANCE_ADD_BAND_4')),
                        ee.Image(img.get('RADIANCE_ADD_BAND_5')),
                        ee.Image(img.get('RADIANCE_ADD_BAND_6')),
                        ee.Image(img.get('RADIANCE_ADD_BAND_7'))]
                        )).toArray(1)

    # create an empty image to save new radiance bands to
    imgArr_OLI = img.select(bands_OLI).toArray().toArray(1);
    Ltoa_OLI = imgArr_OLI.multiply(rad_mult_OLI).add(rad_add_OLI)

    # print(Ltoa_OLI)
     # esun (extra-terrestrial solar irradiance) Units = mW cm-2 um-1
    ESUN_OLI = ee.Image.constant(197.24790954589844).\
      addBands(ee.Image.constant(201.98426818847656)).\
      addBands(ee.Image.constant(186.12677001953125)).\
      addBands(ee.Image.constant(156.95257568359375)).\
      addBands(ee.Image.constant(96.04714965820312)).\
      addBands(ee.Image.constant(23.8833221450863)).\
      addBands(ee.Image.constant(8.04995873449635)).toArray().toArray(1)
    ESUN_OLI = ESUN_OLI.multiply(ee.Image(1))

    ESUNImg_OLI = ESUN_OLI.arrayProject([0]).arrayFlatten([bands_OLI])

    # Ozone Correction
    # Ozone coefficients https://www.arm.gov/publications/tech_reports/doe-sc-arm-tr-129.pdf?id=811 (Appendix A) by band center (lambda)
    koz_OLI = ee.Image.constant(0.0039).addBands(ee.Image.constant(0.0218)).\
      addBands(ee.Image.constant(0.1078)).addBands(ee.Image.constant(0.0608)).addBands(ee.Image.constant(0.0019)).addBands(ee.Image.constant(0)).addBands(ee.Image.constant(0)).toArray().toArray(1)

    # Calculate ozone optical thickness
    Toz_OLI = koz_OLI.multiply(DU_OLI).divide(ee.Image.constant(1000));
    # Calculate TOA radiance in the absense of ozone
    Lt_OLI = Ltoa_OLI.multiply(((Toz_OLI)).multiply((ee.Image.constant(1).divide(cosdSunZe_OLI)).add(ee.Image.constant(1).divide(cosdSatZe_OLI))).exp())

    # Rayleigh optical thickness
    bandCenter_OLI = ee.Image(443).divide(1000).addBands(ee.Image(483).divide(1000)).addBands(ee.Image(561).divide(1000)).addBands(ee.Image(655).divide(1000)).addBands(ee.Image(865).divide(1000)).addBands(ee.Image(1609).divide(1000)).addBands(ee.Number(2201).divide(1000)).toArray().toArray(1)

     # create an empty image to save new Tr values to
    Tr_OLI = (P_OLI.divide(Po_OLI)).multiply(ee.Image(0.008569).multiply(bandCenter_OLI.pow(-4))).multiply((ee.Image(1).add(ee.Image(0.0113).multiply(bandCenter_OLI.pow(-2))).add(ee.Image(0.00013).multiply(bandCenter_OLI.pow(-4)))))

    # Fresnel Reflection
    # Specular reflection (s- and p- polarization states)
    theta_V_OLI = ee.Image(0.0000000001)
    sin_theta_j_OLI = sindSunZe_OLI.divide(ee.Image(1.333))

    theta_j_OLI = sin_theta_j_OLI.asin().multiply(ee.Image(180).divide(ee.Image.constant(math.pi)))

    theta_SZ_OLI = SunZe_OLI

    R_theta_SZ_s_OLI = (((theta_SZ_OLI.multiply(ee.Image.constant(math.pi)
                                                .divide(ee.Image(180)))).subtract(theta_j_OLI.multiply(ee.Image.constant(math.pi).divide(ee.Image(180))))).sin().pow(2)).divide((((theta_SZ_OLI.multiply(ee.Image.constant(math.pi).divide(ee.Image(180)))).add(theta_j_OLI.multiply(ee.Image.constant(math.pi).divide(ee.Image(180))))).sin().pow(2)))

    R_theta_V_s_OLI = ee.Image(0.0000000001)

    R_theta_SZ_p_OLI = (((theta_SZ_OLI.multiply(ee.Image.constant(math.pi).divide(180))).subtract(theta_j_OLI.multiply(ee.Image.constant(math.pi).divide(180)))).tan().pow(2)).divide((((theta_SZ_OLI.multiply(ee.Image.constant(math.pi).divide(180))).add(theta_j_OLI.multiply(ee.Image.constant(math.pi).divide(180)))).tan().pow(2)))

    R_theta_V_p_OLI = ee.Image(0.0000000001)

    R_theta_SZ_OLI = ee.Image(0.5).multiply(R_theta_SZ_s_OLI.add(R_theta_SZ_p_OLI))

    R_theta_V_OLI = ee.Image(0.5).multiply(R_theta_V_s_OLI.add(R_theta_V_p_OLI))

    # Rayleigh scattering phase function
    # Sun-sensor geometry

    theta_neg_OLI = ((cosdSunZe_OLI.multiply(ee.Image(-1))).multiply(cosdSatZe_OLI)).subtract((sindSunZe_OLI).multiply(sindSatZe_OLI).multiply(cosdRelAz_OLI))

    theta_neg_inv_OLI = theta_neg_OLI.acos().multiply(ee.Image(180).divide(ee.Image.constant(math.pi)))

    theta_pos_OLI = (cosdSunZe_OLI.multiply(cosdSatZe_OLI)).subtract(sindSunZe_OLI.multiply(sindSatZe_OLI).multiply(cosdRelAz_OLI))

    theta_pos_inv_OLI = theta_pos_OLI.acos().multiply(ee.Image(180).divide(ee.Image.constant(math.pi)))

    cosd_tni_OLI = theta_neg_inv_OLI.multiply((ee.Image.constant(math.pi)).divide(180)).cos() # in degrees

    cosd_tpi_OLI = theta_pos_inv_OLI.multiply((ee.Image.constant(math.pi)).divide(180)).cos() # in degrees

    Pr_neg_OLI = ee.Image(0.75).multiply((ee.Image(1).add(cosd_tni_OLI.pow(2))))

    Pr_pos_OLI = ee.Image(0.75).multiply((ee.Image(1).add(cosd_tpi_OLI.pow(2))))

    # Rayleigh scattering phase function
    Pr_OLI = Pr_neg_OLI.add((R_theta_SZ_OLI.add(R_theta_V_OLI)).multiply(Pr_pos_OLI));

    # Calulate Lr,
    denom_OLI = ee.Image(4).multiply(ee.Image.constant(math.pi)).multiply(cosdSatZe_OLI)
    Lr_OLI = (ESUN_OLI.multiply(Tr_OLI)).multiply(Pr_OLI.divide(denom_OLI))

    # Rayleigh corrected radiance
    Lrc_OLI = (Lt_OLI.divide(ee.Image(10))).subtract(Lr_OLI)
    LrcImg_OLI = Lrc_OLI.arrayProject([0]).arrayFlatten([bands_OLI])

    # Rayleigh corrected reflectance
    prc_OLI = Lrc_OLI.multiply(ee.Image.constant(math.pi)).multiply(d_OLI.pow(2)).divide(ESUN_OLI.multiply(cosdSunZe_OLI))
    prcImg_OLI = prc_OLI.arrayProject([0]).arrayFlatten([bands_OLI])
    rhorc = prc_OLI.arrayProject([0]).arrayFlatten([bands_OLI])

    # Aerosol Correction
    # Bands in nm
    bands_nm_OLI = ee.Image(443).addBands(ee.Image(483)).addBands(ee.Image(561)).addBands(ee.Image(655)).addBands(ee.Image(865)).addBands(ee.Image(0)).addBands(ee.Image(0)).toArray().toArray(1)

    # Lam in SWIR bands
    Lam_6_OLI = LrcImg_OLI.select('B6')
    Lam_7_OLI = LrcImg_OLI.select('B7')

    # Calculate aerosol type
    eps_OLI = (((((Lam_7_OLI).divide(ESUNImg_OLI.select('B7'))).log()).subtract(((Lam_6_OLI).divide(ESUNImg_OLI.select('B6'))).log())).divide(ee.Image(2201).subtract(ee.Image(1609)))) #.multiply(water_mask)

    # Calculate multiple scattering of aerosols for each band
    Lam_OLI = (Lam_7_OLI).multiply(((ESUN_OLI).divide(ESUNImg_OLI.select('B7')))).multiply((eps_OLI.multiply(ee.Image(-1))).multiply((bands_nm_OLI.divide(ee.Image(2201)))).exp())

    # diffuse transmittance trans_OLI = Tr_OLI.multiply(ee.Image(-1)).divide(ee.Image(2)).multiply(ee.Image(1).divide(cosdSatZe_OLI)).exp()
    trans_OLI = Tr_OLI.multiply(ee.Image(-1)).divide(ee.Image(2)).multiply(ee.Image(1).divide(cosdSatZe_OLI)).exp()

    # Compute water-leaving radiance
    Lw_OLI = Lrc_OLI.subtract(Lam_OLI).divide(trans_OLI)

    # water-leaving reflectance
    pw_OLI = (Lw_OLI.multiply(ee.Image.constant(math.pi)).multiply(d_OLI.pow(2)).divide(ESUN_OLI.multiply(cosdSunZe_OLI)))
    pwImg_OLI = pw_OLI.arrayProject([0]).arrayFlatten([bands_OLI])

    # Rrs
    Rrs = (pw_OLI.divide(ee.Image.constant(math.pi)).arrayProject([0]).arrayFlatten([bands_OLI]).slice(0,5))
    # .multiply(mask)
    Rrs = Rrs.updateMask(Rrs.gt(0));
    
    processed_image = ee.Image(img).addBands(Rrs)
    
    # Optionally, you can set metadata for the processed image if needed
    processed_image = processed_image.set('atmospheric_correction_applied', True)
    
    return processed_image
    # return Rrs.set('system:time_start',img.get('system:time_start'))


# Define Landsat 5 bands and their corresponding standard names
LC5_BANDS = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'ST_B6', 'SR_B7','SZA', 'VZA', 'VAA','SAA']
# L8_BANDS = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'ST_B10', 'SR_B7','SZA', 'VZA', 'VAA','SAA']
L8_BANDS = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'ST_B10', 'SR_B7']
STD_NAMES = ['B1','B2','B3','B4','B5','B6','temp','B7']

# STD_NAMES = ['B1','B2','B3','B4','B5','B6','temp','B7', 'SZA', 'VZA', 'VAA','SAA']

# 'B1','B2','B3','B4','B5','B6','B7'


# Define a function to mask clouds and cloud shadows from Landsat 5 surface reflectance imagery
def maskL457sr(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)
    waterMask = image.select('QA_PIXEL').eq(5440).Or(image.select('QA_PIXEL').eq(5504))
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBand = image.select('ST_B6').multiply(0.00341802).add(-124.15)
    return image.addBands(opticalBands, None, True).addBands(thermalBand, None, True).updateMask(qaMask).updateMask(saturationMask).updateMask(waterMask)

# Define a function to mask clouds and cloud shadows from Landsat 8 surface reflectance imagery
def maskL8sr(image):
    # Define cloud, cloud shadow, and cirrus bitmasks
    cloud_shadow_bit_mask = 1 << 3
    cloud_cirrus_bit_mask = 1 << 2
    clouds_bit_mask = 1 << 4

    # Get the pixel QA band
    qa = image.select('QA_PIXEL')
    # Create masks for cloud, cloud shadow, and cirrus
    cloud_shadow_mask = qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0)
    clouds_mask = qa.bitwiseAnd(clouds_bit_mask).eq(0)
    cirrus_mask = qa.bitwiseAnd(cloud_cirrus_bit_mask).eq(0)

    # Combine masks to filter out cloudy, cloud shadow, and cirrus pixels
    mask = cloud_shadow_mask.And(clouds_mask).And(cirrus_mask)

    # Create saturation mask
    saturation_mask = image.select('QA_RADSAT').eq(0)

    # Apply scaling factors to optical bands
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)

    # Apply scaling factors to thermal bands
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(-124.15)

    # Replace original bands with scaled ones and apply masks
    return image.addBands(optical_bands, None, True) \
                .addBands(thermal_bands, None, True) \
                .updateMask(mask) \
                .updateMask(saturation_mask)

# Define a function to compute the mean and standard deviation of reflectance values for each band
def reflectance(img, lake_point):
    # Define the reducers for mean and standard deviation
    reducer_mean = ee.Reducer.mean()
    reducer_stdDev = ee.Reducer.stdDev()
    
    # Reduce the image to compute mean and standard deviation for each band
    reflectance_values = img.reduceRegion(
        reducer=reducer_mean.combine(reducer2=reducer_stdDev, sharedInputs=True), 
        geometry=lake_point, 
        scale=60
    )

    # Set properties on the image
    img = img.set('DATE_SMP', img.date().format()).set('reflectance', reflectance_values)
    
    # Extract band values and add them as properties
    for band in LC5_BANDS:  # Assuming LC5_BANDS is the list of Landsat 5 bands
        value = img.get(band)
        img = img.set(band, value)
    
    return img

# Define a function to get lake points with buffer
def get_buffered_point(row, buffer_size=100):
    # Create a point geometry using the latitude and longitude values
    lake_point = ee.Geometry.Point(row['Lon'], row['Lat'])
    # Apply buffer to the point
    buffered_point = lake_point.buffer(buffer_size)
    return buffered_point

# Apply the function to the lake points DataFrame
df_coord['Buffered_Point'] = df_coord.apply(lambda row: get_buffered_point(row), axis=1)

# Define a custom Earth Engine function to call get_s2_sr_cld_col
def add_s2_sr_cld_col(img):
    s2_sr_cld_col = get_s2_sr_cld_col(img)
    return img.addBands(s2_sr_cld_col)

# Initialize an empty list to store the filtered DataFrames for each combination of path and row
filtered_dfs = []

# Loop over each path and row combination
for path, row in path_row_combinations:
    # Filter Landsat 5 imagery for the specific path and row
    l5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
        .filter(ee.Filter.eq('WRS_PATH', path)) \
        .filter(ee.Filter.eq('WRS_ROW', row)) \
        .filter(ee.Filter.lt('CLOUD_COVER', 25)) \
        .map(maskL457sr) \
        .filter(ee.Filter.calendarRange(1, 12, 'month')) \
        .select(LC5_BANDS, STD_NAMES)

    # Filter Landsat 8 imagery for the specific path and row
    l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filter(ee.Filter.eq('WRS_PATH', path)) \
        .filter(ee.Filter.eq('WRS_ROW', row)) \
        .filter(ee.Filter.lt('CLOUD_COVER', 25)) \
        .map(maskL8sr) \
        .filter(ee.Filter.calendarRange(1, 12, 'month')) \
        .select(L8_BANDS, STD_NAMES)

    # Merge Landsat 5 and Landsat 8 imagery
    # l = l5.merge(l8)

    

# Map the custom Earth Engine function over the Landsat 5 imagery


# Loop over the buffered points in df_coord DataFrame
    for _, row_coord in df_coord.tail(10).iterrows():
        pondId = row_coord['PERMANENT_ID']
        pondname = row_coord['GNIS_Name']
        lat = row_coord['Lat']  # Extract latitude
        lon = row_coord['Lon']  # Extract longitude
    
        # Get the buffered point geometry
        buffered_point = row_coord['Buffered_Point']
    
        # l8_ATMOSPHERIC = l8.map(lambda img: get_s2_sr_cld_col(img, buffered_point))
        # print("Type of 'l5_ATMOSPHERIC' after map operation:", type(l5_ATMOSPHERIC))
        # l5_with_s2 = l5.map(lambda img: img.addBands(get_s2_sr_cld_col))
    
        # l5_with_s2 = l5.map(add_s2_sr_cld_col)
        # l5_with_s2_first = l5.map(add_s2_sr_cld_col).first()
    

    
        # first_img_2 = l5_ATMOSPHERIC.first()
        # print("Band names of the  l5_ATMOSPHERIC get_s2_sr_cld_col first image:", first_img_2.bandNames().getInfo())
    
        # l5_ATMOSPHERICCorrection = l5_ATMOSPHERIC.map(lambda img: atm_corr(img))  # Fix this line
        # l8_ATMOSPHERICCorrection = l8_ATMOSPHERIC.map(atm_corr)
    
        # first_img_1 = l5_ATMOSPHERICCorrection.first()
        # print("Band names of the  l5_ATMOSPHERICCorrection atm_corr first image:", first_img_1.bandNames().getInfo())
    
        # print("Type of 'l5_ATMOSPHERICCorrection' after map operation:", type(l5_ATMOSPHERICCorrection))
    
        
    
        # Map the reflectance function over the Landsat imagery for the specific path and row
        map_reflectance = l8.map(lambda img: reflectance(img, buffered_point))
        
        # Print band names of the first image in the mapped collection
        # first_img = map_reflectance.first()
        # print("Band names of the first image:", first_img.bandNames().getInfo())
  

        # Reduce the mapped image collection to get reflectance values for the specific path and row
        list_reflectance = map_reflectance.reduceColumns(ee.Reducer.toList(2), ['DATE_SMP', 'reflectance']).values().get(0)
    
        # print(list_reflectance.getInfo())
        # # Map the reflectance function over the Landsat imagery for the specific path and row
        # map_reflectance = l5_ATMOSPHERICCorrection.map(lambda img: reflectance(img, buffered_point))
        
        # # map_reflectance = l.map(lambda img: reflectance(img, buffered_point))
    
    
        
        # # Reduce the mapped image collection to get reflectance values for the specific path and row
        # list_reflectance = map_reflectance.reduceColumns(ee.Reducer.toList(2), ['DATE_SMP', 'reflectance']).values().get(0)
    
        # Convert the results to a pandas DataFrame
        df_reflectance = pd.DataFrame(list_reflectance.getInfo(), columns=['DATE_SMP', 'reflectance'])
        df_reflectance['DATE_SMP'] = pd.to_datetime(df_reflectance['DATE_SMP'])
        df_reflectance['DATE_SMP'] = df_reflectance['DATE_SMP'].dt.date
        df_reflectance['reflectance'] = df_reflectance['reflectance'].apply(
            lambda x: {k: v for k, v in x.items() if v is not None})
    
        # Unpack the 'reflectance' dictionary and create separate columns for each band
        df_reflectance = pd.concat([df_reflectance.drop('reflectance', axis=1),
                                    df_reflectance['reflectance'].apply(pd.Series).astype('float64', errors='ignore')], axis=1)
    
        # Add additional columns including latitude and longitude
        df_reflectance['WRS_PATH'] = path
        df_reflectance['WRS_ROW'] = row
        df_reflectance['POND_ID'] = pondId
        df_reflectance['POND_NAME'] = pondname
        df_reflectance['Latitude'] = lat  # Add latitude
        df_reflectance['Longitude'] = lon  # Add longitude
    
        # Add the DataFrame to the list
        filtered_dfs.append(df_reflectance)


# Concatenate all filtered DataFrames into a single DataFrame
df_filtered_lakes_800 = pd.concat(filtered_dfs, ignore_index=True)
 
# Sort the DataFrame by 'DATE' in ascending order
df_filtered_lakes_800.sort_values(by='DATE_SMP', inplace=True)

# Display or further process the resulting DataFrame
print(df_filtered_lakes_800)



        DATE_SMP   B1_mean  B1_stdDev   B2_mean  B2_stdDev   B3_mean  \
0     2013-05-16  0.009107   0.004419  0.012408   0.005034  0.016294   
1036  2013-05-16       NaN        NaN       NaN        NaN       NaN   
970   2013-05-16       NaN        NaN       NaN        NaN       NaN   
64    2013-05-16  0.006411   0.003005  0.009567   0.004312  0.017964   
904   2013-05-16       NaN        NaN       NaN        NaN       NaN   
...          ...       ...        ...       ...        ...       ...   
1233  2024-02-08       NaN        NaN       NaN        NaN       NaN   
1035  2024-02-08       NaN        NaN       NaN        NaN       NaN   
575   2024-02-08  0.737091   0.002076  0.773920   0.002416  0.798298   
837   2024-02-08  0.483103   0.238461  0.512803   0.252490  0.539721   
1299  2024-02-08       NaN        NaN       NaN        NaN       NaN   

      B3_stdDev   B4_mean  B4_stdDev   B5_mean  ...   B7_mean  B7_stdDev  \
0      0.009198  0.015233   0.011528  0.027190  ...  0.0151